In [ ]:
%run create_topic -t test

In [ ]:
%run produce -t test_docker -n 100 -s 1

In [ ]:
%run consume -t test_docker

# Read from Kafka and insert into Mongo

In [ ]:
from pprint import pprint
from pymongo import MongoClient
from dotenv import load_dotenv
from kafka import KafkaConsumer
from json import loads

import os

load_dotenv()

USER = os.environ.get('MONGO_USER')
PASSWORD = os.environ.get('MONGO_PASSWORD')

client = MongoClient('localhost', port=27017, username=USER, password=PASSWORD)

# get mongo db and collection
db = client['test_db']
test_kafka_collection = db['test_kafka_2']

# server = db.command("serverStatus")
# pprint(server)

# get Kafka messages
consumer = KafkaConsumer(
    'test_docker',
     bootstrap_servers=['localhost:9092'],
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     session_timeout_ms=1000,
     consumer_timeout_ms=1000,
     value_deserializer=lambda x: loads(x.decode('utf-8')))

bulk_insert = []

for m in consumer:
#     print(f"{m}")    
#     print(f"{m.value}")
    value = {
        'time': m.timestamp,
        'number': m.value['number']
    }
    
    bulk_insert.append(value)
    
# insert into collection in mongodb db
test_kafka_collection.insert_many(bulk_insert)

# Print and Consumer Data from Mongo

In [ ]:
from pprint import pprint
from pymongo import MongoClient
from dotenv import load_dotenv

import os

load_dotenv()

USER = os.environ.get('MONGO_USER')
PASSWORD = os.environ.get('MONGO_PASSWORD')

client = MongoClient('localhost', port=27017, username=USER, password=PASSWORD)

# get mongo db and collection
db = client['test_db']
test_kafka_collection = db['test_kafka_2']
print(test_kafka_collection.count_documents({}))
for m in test_kafka_collection.find():
    pprint(m)